In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import ast
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import re
from fuzzywuzzy import process


# Load the DataFrame from a CSV file
reviews = pd.read_csv('/Users/danielebelmiro/Data Analytics Bootcamp/Rotten/reviews_emotions.csv')

In [2]:
reviews

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,predicted_moods
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,"[('excitement', 35.543839830098335), ('approva..."
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,"[('disapproval', 56.05641547237329), ('annoyan..."
2,city_hunter_shinjuku_private_eyes,City Hunter: Shinjuku Private Eyes,2590987,2019-05-28,Reuben Baron,False,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,The choreography is so precise and lifelike at...,"[('admiration', 80.08288365216127), ('approval..."
3,city_hunter_shinjuku_private_eyes,City Hunter: Shinjuku Private Eyes,2558908,2019-02-14,Matt Schley,False,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,The films outoftouch attempts at humor may fin...,"[('amusement', 22.690977576957067), ('realizat..."
4,dangerous_men_2015,Dangerous Men,2504681,2018-08-29,Pat Padua,False,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,Its clumsy determination is endearing and some...,"[('amusement', 49.197768434566136), ('admirati..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1363574,thor_love_and_thunder,Thor: Love and Thunder,102706151,2022-07-05,Christie Cronan,False,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,Solid but not totally sold Thor Ragnarok still...,"[('admiration', 46.784670468845704), ('disappo..."
1363575,thor_love_and_thunder,Thor: Love and Thunder,102706150,2022-07-05,Ian Sandwell,False,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,Thor Love and Thunder is the most entertaining...,"[('admiration', 36.97271463016962), ('amusemen..."
1363576,thor_love_and_thunder,Thor: Love and Thunder,102706149,2022-07-05,Lauren LaMagna,False,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,Thor Love and Thunder is a stepup from Thor Ra...,"[('approval', 51.977043591835596), ('love', 18..."
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,"[('disappointment', 31.922172596719218), ('ann..."


In [3]:
movies = pd.read_csv('/Users/danielebelmiro/Data Analytics Bootcamp/Rotten/clean_movies.csv')

In [4]:
movies

,id,title,audienceScore,tomatoMeter,runtimeMinutes,genre,originalLanguage,director,writer,release_year
0,love_lies,"Love, Lies",43.00,65.76,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown
1,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Fantasy,English,Will Meugniot,John Loy,2017
2,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Adventure,English,Will Meugniot,John Loy,2017
3,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Animation,English,Will Meugniot,John Loy,2017
4,adrift_2018,Adrift,65.00,69.00,93.0,Adventure,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018
...,...,...,...,...,...,...,...,...,...,...
117591,fun_size,Fun Size,47.00,25.00,86.0,Holiday,English,Josh Schwartz,Max Werner,2012
117592,fun_size,Fun Size,47.00,25.00,86.0,Comedy,English,Josh Schwartz,Max Werner,2012
117593,dassehra,Dassehra,55.67,65.76,131.0,Action,Hindi,Manish Vatsalya,Saurabh Choudhary,2019
117594,dassehra,Dassehra,55.67,65.76,131.0,Thriller,Hindi,Manish Vatsalya,Saurabh Choudhary,2019


In [26]:
movies['id'].nunique()

75560

#### Calculating the average emotions per movie:

### Explode and Split the Data
First, explode and split the predicted_moods column into emotion and probability:

In [5]:
# Convert tuple strings into actual tuples
reviews['predicted_moods'] = reviews['predicted_moods'].apply(ast.literal_eval)

# Explode the column
reviews_exploded = reviews.explode('predicted_moods')

# Split the tuples into separate columns
reviews_exploded[['emotion', 'probability']] = pd.DataFrame(reviews_exploded['predicted_moods'].tolist(), index=reviews_exploded.index)
reviews_exploded = reviews_exploded.drop(columns=['predicted_moods'])

# Convert 'probability' to numeric
reviews_exploded['probability'] = pd.to_numeric(reviews_exploded['probability'])

# Verify the result
reviews_exploded

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,excitement,35.543840
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,approval,33.460199
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,admiration,14.496543
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,disapproval,56.056415
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,annoyance,16.554233
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,annoyance,29.755378
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,disapproval,16.977576
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,amusement,66.380384
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,joy,22.703058


### Normalize Probabilities to Sum to 100%

In [6]:
# Normalize probabilities to sum to 100%
sum_probabilities = reviews_exploded.groupby('reviewId')['probability'].transform('sum')
reviews_exploded['probability_normalized'] = (reviews_exploded['probability'] / sum_probabilities) * 100

# Round the normalized probabilities to 4 decimal places
reviews_exploded['probability_normalized'] = reviews_exploded['probability_normalized'].round(4)

# Verify the result
reviews_exploded

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability,probability_normalized
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,excitement,35.543840,42.5672
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,approval,33.460199,40.0718
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,admiration,14.496543,17.3610
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,disapproval,56.056415,70.6446
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,annoyance,16.554233,20.8623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,annoyance,29.755378,37.8302
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,disapproval,16.977576,21.5848
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,amusement,66.380384,71.3199
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,joy,22.703058,24.3925


In [7]:
# Verify the sum of normalized probabilities
sum_normalized = reviews_exploded.groupby('reviewId')['probability_normalized'].sum()

# Round the sums to 2 decimal places for display
sum_normalized = sum_normalized.round(2)

# Check unique values
print(sum_normalized.unique())

[100.]


## group by id and emotion and calculate the mean probability

In [8]:
# Agrupar por 'id' e 'emotion', calculando a média das probabilidades
reviews_grouped = reviews_exploded.groupby(['id', 'emotion'])['probability_normalized'].mean().reset_index()

# Verify the result
reviews_grouped.head(10)

,id,emotion,probability_normalized
0,$5_a_day,admiration,43.15200
1,$5_a_day,amusement,33.83000
2,$5_a_day,annoyance,36.67500
3,$5_a_day,approval,34.83245
4,$5_a_day,disappointment,32.67010
5,$5_a_day,disapproval,50.40925
6,$5_a_day,joy,21.32340
7,$5_a_day,optimism,11.22480
8,$5_a_day,realization,7.48920
9,009_re_cyborg,admiration,49.29476


In [9]:
pd.set_option('display.max_colwidth', None)

reviews_exploded.loc[reviews_exploded['id'] == '$5_a_day']

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability,probability_normalized
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,disapproval,60.371897,70.1637
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,approval,19.228371,22.3471
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,realization,6.444050,7.4892
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,annoyance,24.614549,36.6750
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,disappointment,21.926653,32.6701
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,disapproval,20.574082,30.6548
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,approval,37.784600,47.3178
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,admiration,33.104835,41.4574
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,optimism,8.963305,11.2248
1213417,$5_a_day,Five Dollars a Day,1780984,2008-11-16,David Nusair,False,fresh,Reel Film Reviews,...very amusing and agreeable...,POSITIVE,very amusing and agreeable,admiration,40.276130,44.8466


In [10]:
# Selecionar as 3 emoções com as maiores médias para cada filme
top_3_emotions = reviews_grouped.groupby('id').apply(
    lambda x: x.nlargest(3, 'probability_normalized')
).reset_index(drop=True)

In [11]:
top_3_emotions

,id,emotion,probability_normalized
0,$5_a_day,disapproval,50.40925
1,$5_a_day,admiration,43.15200
2,$5_a_day,annoyance,36.67500
3,009_re_cyborg,disapproval,56.85230
4,009_re_cyborg,annoyance,50.76375
...,...,...,...
202807,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,approval,58.79150
202808,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,fear,33.92220
202809,zz_top_that_little_ol_band_from_texas,amusement,57.87560
202810,zz_top_that_little_ol_band_from_texas,joy,56.15785


In [12]:
# Normalizar as probabilidades para somar 100% para cada filme
top_3_emotions['probability_normalized'] = top_3_emotions.groupby('id')['probability_normalized'].transform(
    lambda x: (x / x.sum()) * 100
)

# Exibir o resultado
print(top_3_emotions.head())

              id      emotion  probability_normalized
0       $5_a_day  disapproval               38.706005
1       $5_a_day   admiration               33.133632
2       $5_a_day    annoyance               28.160362
3  009_re_cyborg  disapproval               36.023280
4  009_re_cyborg    annoyance               32.165396


In [18]:
top_3_emotions

,id,emotion,probability_normalized
0,$5_a_day,disapproval,38.706005
1,$5_a_day,admiration,33.133632
2,$5_a_day,annoyance,28.160362
3,009_re_cyborg,disapproval,36.023280
4,009_re_cyborg,annoyance,32.165396
...,...,...,...
202807,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,approval,38.438228
202808,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,fear,22.178533
202809,zz_top_that_little_ol_band_from_texas,amusement,35.673589
202810,zz_top_that_little_ol_band_from_texas,joy,34.614795


In [20]:
# Agrupar por 'id' e criar a lista de tuplas (emotion, probability_normalized)
new_df = top_3_emotions.groupby('id').apply(
    lambda x: list(zip(x['emotion'], x['probability_normalized'])))
new_df = new_df.reset_index()
new_df.columns = ['id', 'emotions']

new_df

,id,emotions
0,$5_a_day,"[(disapproval, 38.70600543243529), (admiration, 33.1336321492672), (annoyance, 28.16036241829752)]"
1,009_re_cyborg,"[(disapproval, 36.023279538211014), (annoyance, 32.165396240044096), (confusion, 31.81132422174489)]"
2,00_mhz,"[(annoyance, 41.840924718059824), (admiration, 36.49896509094363), (joy, 21.660110190996555)]"
3,1,"[(admiration, 61.524699999999996), (approval, 35.5203), (love, 2.955)]"
4,1-day,"[(admiration, 41.199610661687295), (amusement, 32.26985631655974), (disappointment, 26.530533021752966)]"
...,...,...
67599,zus_and_zo_2003,"[(desire, 40.01421221408237), (amusement, 30.742439443586793), (joy, 29.243348342330837)]"
67600,zvenigora,"[(confusion, 71.48010000000001), (disapproval, 17.093200000000003), (annoyance, 11.426700000000002)]"
67601,zwei_mutter_2013,"[(approval, 47.60089193324174), (admiration, 30.408156407361503), (realization, 21.990951659396757)]"
67602,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,"[(sadness, 39.38323877121109), (approval, 38.43822776765831), (fear, 22.178533461130584)]"


In [22]:
# merge entre movies e new_df

merged_df = movies.merge(new_df, on='id', how='left')

merged_df

,id,title,audienceScore,tomatoMeter,runtimeMinutes,genre,originalLanguage,director,writer,release_year,title_normalized,emotions
0,love_lies,"Love, Lies",43.00,65.76,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,love lies,"[(admiration, 78.20800919346894), (joy, 10.896268781721222), (approval, 10.89572202480984)]"
1,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Fantasy,English,Will Meugniot,John Loy,2017,dinosaur island,NaN
2,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Adventure,English,Will Meugniot,John Loy,2017,dinosaur island,NaN
3,dinosaur_island_2002,Dinosaur Island,70.00,65.76,80.5,Animation,English,Will Meugniot,John Loy,2017,dinosaur island,NaN
4,adrift_2018,Adrift,65.00,69.00,93.0,Adventure,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018,adrift,"[(admiration, 42.079146438693414), (disappointment, 29.810255887131987), (approval, 28.1105976741746)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
117591,fun_size,Fun Size,47.00,25.00,86.0,Holiday,English,Josh Schwartz,Max Werner,2012,fun size,"[(confusion, 48.71943200404655), (disappointment, 26.36692449980985), (amusement, 24.913643496143578)]"
117592,fun_size,Fun Size,47.00,25.00,86.0,Comedy,English,Josh Schwartz,Max Werner,2012,fun size,"[(confusion, 48.71943200404655), (disappointment, 26.36692449980985), (amusement, 24.913643496143578)]"
117593,dassehra,Dassehra,55.67,65.76,131.0,Action,Hindi,Manish Vatsalya,Saurabh Choudhary,2019,dassehra,"[(approval, 71.40657548299487), (admiration, 20.783216447748366), (annoyance, 7.810208069256784)]"
117594,dassehra,Dassehra,55.67,65.76,131.0,Thriller,Hindi,Manish Vatsalya,Saurabh Choudhary,2019,dassehra,"[(approval, 71.40657548299487), (admiration, 20.783216447748366), (annoyance, 7.810208069256784)]"


In [23]:
reviews[reviews['id'] == 'dinosaur_island_2002']

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,predicted_moods


### Transformar os Dados para o Formato Wide


In [13]:
# Transformar o DataFrame para o formato wide
reviews_wide = top_3_emotions.pivot(index='id', columns='emotion', values='probability_normalized')

# Preencher valores ausentes com 0
reviews_wide = reviews_wide.fillna(0)

# Exibir o DataFrame transformado
reviews_wide.head()

emotion,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
id,,,,,,,,,,,,,,,,,,,,,
$5_a_day,33.133632,0.000000,0.0,28.160362,0.0000,0.0,0.000000,0.0,0.0,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009_re_cyborg,0.000000,0.000000,0.0,32.165396,0.0000,0.0,31.811324,0.0,0.0,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00_mhz,36.498965,0.000000,0.0,41.840925,0.0000,0.0,0.000000,0.0,0.0,0.000000,...,21.66011,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,61.524700,0.000000,0.0,0.000000,35.5203,0.0,0.000000,0.0,0.0,0.000000,...,0.00000,2.955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-day,41.199611,32.269856,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.0,26.530533,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Cálculo de similaridade

normalização dos nomes dos filmes e a busca aproximada usando a biblioteca fuzzywuzzy:

In [14]:
# Function to normalize names
def normalize_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Remove special characters
    name = name.lower().strip()  # Convert to lowercase and remove extra spaces
    return name

In [15]:
# Function to find the closest matching movie
def find_closest_movie(movie_name, sim_df):
    # Normalize the movie name
    movie_name = normalize_name(movie_name)
    
    # Find the closest match
    match, score = process.extractOne(movie_name, sim_df.index)
    
    if score >= 80:  # Adjust the similarity threshold as needed
        return match  # Return the matched title
    else:
        return None

uso de PCA (Principal Component Analysis) para reduzir a dimensionalidade dos dados antes de calcular a similaridade:

In [16]:
# Function to calculate similarity matrix based on emotion data
def calculate_similarity_matrix(reviews_wide):
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=0.95)  # Keep 95% of the variance
    reviews_reduced = pca.fit_transform(reviews_wide.values)
    
    # Calculate cosine similarity
    sim_matrix = cosine_similarity(reviews_reduced)
    
    # Convert to DataFrame
    sim_df = pd.DataFrame(sim_matrix, index=reviews_wide.index, columns=reviews_wide.index)
    
    return sim_df

## Main recommendation function

In [17]:
# Main recommendation function
def recommend_similar_movies(sim_df, movies, reviews, top_n=5):
    # Normalize the titles in the movies DataFrame
    movies['title_normalized'] = movies['title'].apply(normalize_name)
    
    while True:
        # Ask for the user's favorite movie
        favorite_movie = input("Enter the name of your favorite movie (or type 'exit' to quit): ").strip()

        # Allow the user to exit
        if favorite_movie.lower() == 'exit':
            print("Exiting the recommendation system. Goodbye!")
            return None

        # Find the closest matching movie in the similarity DataFrame
        favorite_movie_title = find_closest_movie(favorite_movie, sim_df)

        if favorite_movie_title is None:
            print(f"The movie '{favorite_movie}' was not found. Please check the name and try again.")
            continue  # Ask for input again
        else:
            print(f"Found movie: {favorite_movie_title}")
            break  # Exit the loop if a valid movie is found

    # Get the similarities for the favorite movie
    movie_similarities = sim_df[favorite_movie_title]

    # Sort movies by similarity (from most to least similar)
    movie_similarities = movie_similarities.sort_values(ascending=False)

    # Remove the favorite movie itself from the recommendations
    if favorite_movie_title in movie_similarities.index:
        movie_similarities = movie_similarities.drop(favorite_movie_title)

    # Filter out movies with negative reviews
    # 1. Calculate the ratio of negative reviews for each movie
    negative_reviews_ratio = reviews[reviews['scoreSentiment'] == 'NEGATIVE'].groupby('id').size() / reviews.groupby('id').size()
    movies_with_high_negative_reviews = negative_reviews_ratio[negative_reviews_ratio > 0.5].index

    # 2. Filter movies based on tomatoMeter < 30, audienceScore < 30, and high negative reviews
    filtered_movies = movies[
        (movies['tomatoMeter'] < 30) & 
        (movies['audienceScore'] < 30) & 
        (movies['id'].isin(movies_with_high_negative_reviews))
    ]

    # Get the IDs of movies to exclude
    movies_to_exclude = filtered_movies['id'].unique()

    # Exclude these movies from the recommendations
    movie_similarities = movie_similarities[~movie_similarities.index.isin(movies_to_exclude)]

    # Get the genres of the favorite movie
    favorite_movie_id = movies[movies['title_normalized'] == favorite_movie_title]['id'].values[0]
    favorite_movie_genres = movies[movies['id'] == favorite_movie_id]['genre'].unique()

    # Filter movies by genre (only recommend movies that share at least one genre with the favorite movie)
    movies_in_same_genre = movies[movies['genre'].isin(favorite_movie_genres)]['title_normalized'].unique()
    movie_similarities = movie_similarities[movie_similarities.index.isin(movies_in_same_genre)]

    # Check if there are enough movies to recommend
    if len(movie_similarities) < top_n:
        print(f"Warning: Only {len(movie_similarities)} similar movies were found.")

    # Select the top_n most similar movies
    recommended_movies = movie_similarities.head(top_n).index.tolist()

    # Fetch additional information from the movies DataFrame
    movie_details = movies[movies['title_normalized'].isin(recommended_movies)]

    # Combine similarity information with movie details
    similarity_df = movie_similarities.head(top_n).reset_index()

    # Check the name of the index column
    if 'index' in similarity_df.columns:
        similarity_df = similarity_df.rename(columns={'index': 'title_normalized'})
    else:
        # If the index has a different name, rename it to 'title_normalized'
        similarity_df = similarity_df.rename(columns={similarity_df.columns[0]: 'title_normalized'})

    # Rename the similarity column to 'similarity'
    similarity_df = similarity_df.rename(columns={similarity_df.columns[1]: 'similarity'})

    # Merge with the correct column
    result = movie_details.merge(
        similarity_df,
        left_on='title_normalized',
        right_on='title_normalized'
    )

    # Add predicted_moods to the result
    # Group reviews by movie ID and combine predicted_moods
    predicted_moods = reviews.groupby('id')['predicted_moods'].first().reset_index()
    result = result.merge(predicted_moods, on='id', how='left')
    
    # Group movies by ID and combine information
    result = result.groupby('id').agg({
        'title': 'first',  # O título é o mesmo para todas as linhas do mesmo filme
        'director': 'first',  # O diretor é o mesmo para todas as linhas do mesmo filme
        'originalLanguage': 'first',  # O idioma é o mesmo para todas as linhas do mesmo filme
        'runtimeMinutes': 'first',  # A duração é a mesma para todas as linhas do mesmo filme
        'genre': lambda x: ', '.join(set(x.dropna())),  # Combina gêneros únicos
        'release_year': 'first',  # O ano de lançamento é o mesmo para todas as linhas do mesmo filme
        'tomatoMeter': 'first',  # O Tomatometer é o mesmo para todas as linhas do mesmo filme
        'audienceScore': 'first',  # A pontuação do público é a mesma para todas as linhas do mesmo filme
        'similarity': 'first',  # A similaridade é a mesma para todas as linhas do mesmo filme
        'predicted_moods': 'first'  # As emoções previstas são as mesmas para todas as linhas do mesmo filme
    }).reset_index()
    
    # Remove duplicates based on 'id'
    result = result.drop_duplicates(subset=['id'])
    
    # Select relevant columns
    result = result[['id', 'title', 'director', 'originalLanguage', 'runtimeMinutes', 'genre', 'release_year', 'tomatoMeter', 'audienceScore', 'similarity', 'predicted_moods']]

    # Format the output for the user
    print(f"\nRecommendations based on the movie '{favorite_movie_title}':\n")
    for _, row in result.iterrows():
        print(f"Movie: {row['title']}")
        print(f"  - Director: {row['director']}")
        print(f"  - Language: {row['originalLanguage']}")
        print(f"  - Duration: {row['runtimeMinutes']} minutes")
        print(f"  - Genre: {row['genre']}")
        print(f"  - Year: {row['release_year']}")
        print(f"  - Tomatometer: {row['tomatoMeter']}%")
        print(f"  - Audience Score: {row['audienceScore']}%")
        print(f"  - Similarity: {row['similarity']:.2f}")
        if isinstance(row['predicted_moods'], list):
            print(f"  - Predicted Moods: {', '.join([f'{mood} ({percentage:.1f}%)' for mood, percentage in row['predicted_moods']])}")
        else:
            print(f"  - Predicted Moods: None")
        print("-" * 40)

    return result

# Example usage
sim_df = calculate_similarity_matrix(reviews_wide)

In [18]:
# Map movie IDs to titles
# Create a dictionary to map IDs to titles
id_to_title = dict(zip(movies['id'], movies['title']))

# Normalize the titles in the dictionary
id_to_title_normalized = {k: normalize_name(v) for k, v in id_to_title.items()}

# Replace IDs with normalized titles in sim_df
sim_df.index = sim_df.index.map(id_to_title_normalized)
sim_df.columns = sim_df.columns.map(id_to_title_normalized)

# Generate recommendations
recommendations = recommend_similar_movies(sim_df, movies, reviews, top_n=5)

Enter the name of your favorite movie (or type 'exit' to quit): five dollars a day
Found movie: five dollars a day

Recommendations based on the movie 'five dollars a day':

Movie: Five
  - Director: Abbas Kiarostami
  - Language: Unknown
  - Duration: 76.16666666666667 minutes
  - Genre: Documentary
  - Year: Unknown
  - Tomatometer: 65.76%
  - Audience Score: 55.67%
  - Similarity: 0.98
  - Predicted Moods: None
----------------------------------------
Movie: Five
  - Director: Arch Oboler
  - Language: English
  - Duration: 76.16666666666667 minutes
  - Genre: Sci-fi
  - Year: 2009
  - Tomatometer: 73.0%
  - Audience Score: 52.0%
  - Similarity: 0.98
  - Predicted Moods: admiration (56.1%), approval (32.0%), realization (5.8%)
----------------------------------------
Movie: Five
  - Director: Jennifer Aniston,Alicia Keys,Demi Moore,Patty Jenkins,Penelope Spheeris
  - Language: English
  - Duration: 76.16666666666667 minutes
  - Genre: Drama, Comedy
  - Year: 2012
  - Tomatometer: 65

In [19]:
recommendations

,id,title,director,originalLanguage,runtimeMinutes,genre,release_year,tomatoMeter,audienceScore,similarity,predicted_moods
0,771459240,Five,Abbas Kiarostami,Unknown,76.166667,Documentary,Unknown,65.76,55.67,0.979365,None
1,five,Five,Arch Oboler,English,76.166667,Sci-fi,2009,73.00,52.00,0.979365,"[(admiration, 56.10576767531564), (approval, 32.0045622362987), (realization, 5.806738266616077)]"
2,five_2011,Five,"Jennifer Aniston,Alicia Keys,Demi Moore,Patty Jenkins,Penelope Spheeris",English,76.166667,"Drama, Comedy",2012,65.76,75.00,0.979365,None
3,five_2016,Five,Igor Gotesman,French,76.166667,Drama,Unknown,65.76,63.00,0.979365,"[(amusement, 76.41798652168731), (joy, 15.692593708736835), (approval, 2.41665787069883)]"
4,five_2020,Five,Jeffrey DeChausse,English,76.166667,Drama,Unknown,65.76,55.67,0.979365,None
5,handsome_a_netflix_mystery_movie,Handsome: A Netflix Mystery Movie,Jeff Garlin,English,103.000000,Comedy,2017,65.76,31.00,0.986499,"[(approval, 29.21198029040348), (amusement, 27.455820837918175), (annoyance, 7.779752888370332)]"
6,hotel_de_love,Hotel de Love,Craig Rosenberg,English,95.000000,"Romance, Comedy",Unknown,22.00,54.00,0.995625,"[(annoyance, 43.69491481024574), (amusement, 17.05829860843535), (disappointment, 10.008292192860333)]"
7,i_love_melvin,I Love Melvin,Don Weis,English,76.000000,"Comedy, Musical",1953,65.76,58.00,0.986386,"[(approval, 35.76052164562983), (realization, 8.843903464406056), (annoyance, 8.385574621844924)]"
8,simple_men,Simple Men,Hal Hartley,English,105.000000,"Romance, Drama, Comedy",2004,91.00,89.00,0.986926,"[(annoyance, 33.98189214639912), (amusement, 20.978656781915454), (disappointment, 6.337381933644022)]"


In [20]:
sim_df.head()

id,five dollars a day,009 re cyborg,00 mhz,1,1 day,1,10,10 violent women,12 angry men,rat,...,zubaan,zulfiqar,zulu,zulu,zulu dawn,hotel paraiso,zvenigora,two mothers,zycie jako smiertelna choroba przenoszona droga plciowa,zz top that little ol band from texas
id,,,,,,,,,,,,,,,,,,,,,
five dollars a day,1.000000,0.641910,0.547113,0.056409,0.013593,0.763169,-0.001634,0.079867,-0.029517,-0.253461,...,-0.026758,0.138734,0.019327,0.270363,-0.233877,-0.168894,0.152330,-0.267019,-0.359125,-0.401159
009 re cyborg,0.641910,1.000000,0.288818,-0.476281,-0.306145,0.751342,0.264273,0.199067,-0.197512,-0.040599,...,0.246710,0.315416,0.255904,-0.350793,-0.393885,0.025428,0.766908,-0.402770,-0.110379,-0.134771
00 mhz,0.547113,0.288818,1.000000,0.131923,0.067086,0.096132,0.033280,-0.195939,0.006196,-0.083593,...,-0.010192,0.289912,0.048438,0.342555,-0.187472,0.043531,0.007051,-0.222559,-0.360611,-0.097552
1,0.056409,-0.476281,0.131923,1.000000,0.167920,-0.435195,0.023473,-0.466734,-0.011479,-0.563865,...,-0.123584,-0.500167,0.012297,0.855898,0.583346,-0.472233,-0.352854,0.601463,-0.158839,-0.320561
1 day,0.013593,-0.306145,0.067086,0.167920,1.000000,-0.164375,0.441717,0.254898,0.474965,0.490813,...,-0.077994,0.147707,0.000432,0.450016,-0.283299,0.090054,-0.219475,-0.324417,-0.533145,-0.045773


In [17]:
# Filtrar filmes com predicted_moods igual a None
movies_without_moods = recommendations[recommendations['predicted_moods'].isna()]

# Exibir os filmes filtrados
if not movies_without_moods.empty:
    print("Filmes sem predicted_moods:")
    print(movies_without_moods[['id', 'title', 'predicted_moods']])
else:
    print("Nenhum filme sem predicted_moods encontrado.")

Filmes sem predicted_moods:
          id title predicted_moods
0  771459240  Five            None
2  five_2011  Five            None
4  five_2020  Five            None


In [28]:
movies.loc[movies['title_normalized'] == "five"]

,id,title,audienceScore,tomatoMeter,runtimeMinutes,genre,originalLanguage,director,writer,release_year,title_normalized
6754,five,Five,52.00,73.00,76.166667,Sci-fi,English,Arch Oboler,Unknown,2009,five
16666,five_2020,Five,55.67,65.76,76.166667,Drama,English,Jeffrey DeChausse,Jeffrey DeChausse,Unknown,five
53983,five_2011,Five,75.00,65.76,76.166667,Comedy,English,"Jennifer Aniston,Alicia Keys,Demi Moore,Patty Jenkins,Penelope Spheeris",Unknown,2012,five
53984,five_2011,Five,75.00,65.76,76.166667,Drama,English,"Jennifer Aniston,Alicia Keys,Demi Moore,Patty Jenkins,Penelope Spheeris",Unknown,2012,five
66026,771459240,Five,55.67,65.76,76.166667,Documentary,Unknown,Abbas Kiarostami,Abbas Kiarostami,Unknown,five
84031,five_2016,Five,63.00,65.76,76.166667,Drama,French,Igor Gotesman,Igor Gotesman,Unknown,five
